import 程式庫


In [1]:
from database import Database
from get_data import Data
from seperate_company import SeperateCompany
from plot import Plot
from backtest import *
from datetime import datetime
from combinations import sim_conditions
from report import Report
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from dataframe import CustomDataFrame

ImportError: cannot import name 'SeperateCompany' from 'seperate_company' (c:\OneDrive\OneDrive - 國立中央大學\許智誠Lab\因子分析回測系統\stock_factor_lab\seperate_company.py)

In [ ]:
# 建立取得資料物件
data = Data()
# 建立資料分割物件
seperate_company = SeperateCompany()
# 建立畫圖物件
plotter = Plot()

In [ ]:
# 取得收盤價資料(回測用)
close = data.get("price:close")
# 取得RSI指標資料(進出場策略用)
rsi = data.indicator("rsi")
# 取得ROE財報資料(因子分析/進出場策略用)
roe = data.get("report:roe")
pb = data.get("report:pb")

In [ ]:
# 利用ROE因子將公司切分成四等分(因子分析)
roe_1, roe_2, roe_3, roe_4 = roe.divide_slice(quantile=4, ascending=False)

In [ ]:
# # 利用PB因子將公司切分成四等分(因子分析)
# pb_1, pb_2, pb_3, pb_4 = pd.divide_slice(quantile=4, ascending=True)

In [ ]:
# 進行遮罩  5 MASK TRUE = 5,  3 MASK FALSE = NAN
# 雙因子會用到
def MASK(df_bool, df_numeric):
    # 使用np.where進行遮罩操作
    result = np.where(df_bool, df_numeric, np.nan)

    # 將結果添加到新的DataFrame中
    result_df = pd.DataFrame(result, columns=df_numeric.columns)

    return CustomDataFrame(result_df)

In [ ]:
# Achieving Alpha 雙因子概念實作
# 以抓出Quantile_1為例：
roe_1_mask_pb = MASK(roe_1, pb)
# 抓出Quantile_4
roe_4_mask_pb = MASK(roe_4, pb)

In [ ]:
# 在用第二個因子PB來切成四等份(roe1)
quantile_roe1_pb1, quantile_roe1_pb2, quantile_roe1_pb3, quantile_roe1_pb4= roe_1_mask_pb.divide_slice(quantile=4, ascending=True)
# 在用第二個因子PB來切成四等份(roe4)
quantile_roe4_pb1, quantile_roe4_pb2, quantile_roe4_pb3, quantile_roe4_pb4= roe_4_mask_pb.divide_slice(quantile=4, ascending=True)

In [ ]:
# 設定進場的條件
cond_1 = rsi.is_largest(5)
cond_2 = close > close.average(20)
# 進場條件做AND運算
cond = cond_1 & cond_2

In [7]:
tmp = {'a1':1,'b1':2}

In [9]:
tmp['a1']

1

In [12]:
k = 'a'
tmp[k+'1']

1

In [ ]:
# 雙因子的Q1、Q4 --> position
postion_1 = quantile_roe1_pb1 & cond
postion_4 = quantile_roe4_pb4 & cond

In [16]:
i = 3

In [17]:
tmp_str = "Quantile_"+(i + 1)+"_MASK_factor2"


TypeError: can only concatenate str (not "int") to str

In [15]:
tmp_str

('Quantile_', 4, '_MASK_factor2')

In [ ]:
# # # 設定postion組合，結合因子分析&進場條件
# postion_1 = roe_1 & cond
# postion_2 = roe_2 & cond
# postion_3 = roe_3 & cond
# postion_4 = roe_4 & cond

In [ ]:
# 將四個區間帶入回測，換股周期為:月
# result_1_M = sim(postion_1, resample="M")
# result_2_M = sim(postion_2, resample="M")
# result_3_M = sim(postion_3, resample="M")
# result_4_M = sim(postion_4, resample="M")

In [ ]:
# 將四個區間帶入回測，換股周期為:季
result_1_Q = sim(postion_1, resample="Q")
# result_2_Q = sim(postion_2, resample="Q")
# result_3_Q = sim(postion_3, resample="Q")
result_4_Q = sim(postion_4, resample="Q")

In [ ]:
# # 創建一個圖表和子圖
# fig, ax = plt.subplots()

# # 在第一個子圖中繪製第一個 DataFrame 的 "cum_returns" 列
# result_1_M.stock_data["cum_returns"].plot(ax=ax, label="result_roe_1_M")
# # 在第一個子圖中繪製第二個 DataFrame 的 "cum_returns" 列
# result_2_M.stock_data["cum_returns"].plot(ax=ax, label="result_roe_2_M")
# # 在第一個子圖中繪製第三個 DataFrame 的 "cum_returns" 列
# result_3_M.stock_data["cum_returns"].plot(ax=ax, label="result_roe_3_M")
# # 在第一個子圖中繪製第四個 DataFrame 的 "cum_returns" 列
# result_4_M.stock_data["cum_returns"].plot(ax=ax, label="result_roe__M")
# # 添加圖例
# ax.legend()

# # 顯示圖表
# plt.show()

In [ ]:
# 創建一個圖表和子圖
fig, ax = plt.subplots()

# 在第一個子圖中繪製第一個 DataFrame 的 "cum_returns" 列
result_1_Q.stock_data["cum_returns"].plot(ax=ax, label="result_roe_1_Q")
# # 在第一個子圖中繪製第二個 DataFrame 的 "cum_returns" 列
# result_2_Q.stock_data["cum_returns"].plot(ax=ax, label="result_roe_2_Q")
# # 在第一個子圖中繪製第三個 DataFrame 的 "cum_returns" 列
# result_3_Q.stock_data["cum_returns"].plot(ax=ax, label="result_roe_3_Q")
# 在第一個子圖中繪製第四個 DataFrame 的 "cum_returns" 列
result_4_Q.stock_data["cum_returns"].plot(ax=ax, label="result_roe_4_Q")
# 添加圖例
ax.legend()

# 顯示圖表
plt.show()

In [ ]:
result_3.display()

In [ ]:
result_2.display()

In [ ]:
result_3.display()

In [ ]:
result_4.display()

In [ ]:
# 實做看看achieving alpha